In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from get_data import get_req_data
import get_tickers
import yahooquery as yq
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# Param configuration
start_date = "2015-01-08"
end_date = "2015-01-15"

pd.set_option('display.max_rows', 20)

#Sector Map
sector_map = {
'S&P 500 Materials (Sector)' : 'basic_materials', 
'S&P 500 Industrials (Sector)' : 'industrials',
'S&P 500 Consumer Discretionary (Sector)' : 'consumer_cyclical' ,
'S&P 500 Consumer Staples (Sector)' : 'consumer_defensive',
'S&P 500 Health Care (Sector)' : 'healthcare', 
'S&P 500 Financials (Sector)' : 'financial_services',
'S&P 500 Information Technology (Sector)' : 'technology',
'S&P 500 Telecommunication Services (Sector)' : 'communication_services',
'S&P 500 Utilities (Sector)' : 'utilities', 
'S&P 500 Real Estate (Sector)' : 'realestate',
'S&P 500 Energy (Sector)' : 'energy'
}


In [ ]:
# get_tickers.main()

In [ ]:
yf.download("^GSPE", start=start_date, end=end_date, progress=False)

In [3]:
get_req_data(start_date, end_date)

{'S&P 500 Materials (Sector)': {'sector_data': Date
  2015-01-07    298.519989
  2015-01-08    305.600006
  2015-01-09    304.070007
  2015-01-12    303.130005
  2015-01-13    299.609985
  2015-01-14    296.049988
  Name: Adj Close, dtype: float64},
 'S&P 500 Industrials (Sector)': {'sector_data': Date
  2015-01-07    471.709991
  2015-01-08    481.109985
  2015-01-09    475.869995
  2015-01-12    472.730011
  2015-01-13    471.809998
  2015-01-14    469.109985
  Name: Adj Close, dtype: float64},
 'S&P 500 Consumer Discretionary (Sector)': {'sector_data': Date
  2015-01-07    560.500000
  2015-01-08    568.830017
  2015-01-09    562.340027
  2015-01-12    559.679993
  2015-01-13    559.020020
  2015-01-14    552.580017
  Name: Adj Close, dtype: float64},
 'S&P 500 Consumer Staples (Sector)': {'sector_data': Date
  2015-01-07    502.549988
  2015-01-08    510.149994
  2015-01-09    506.049988
  2015-01-12    504.309998
  2015-01-13    504.239990
  2015-01-14    502.910004
  Name: Adj Cl

In [17]:
# Generate data frame to hold all required information
sectors_ticker = "SPY"

# Load in info
sp500_tickers_data = get_req_data(start_date, end_date)
weights = yq.Ticker(sectors_ticker).fund_sector_weightings.to_dict()

# add the sectors + dates + adj close to the dataframe
records = []
for sector, data in sp500_tickers_data.items():
    for x in range(0, len(data['sector_data'])):

        record = {"sector" : sector, "date" : data['sector_data'].index[x], "adj_close" : data["sector_data"][x], "sector_weight" : weights[sectors_ticker][sector_map[sector]] }
        records.append(record)

df = pd.DataFrame(records)

df["pct_change"] = df.groupby("sector")["adj_close"].pct_change()

df["contribution"] = df["pct_change"] * df["sector_weight"]

print(weights)
display(df)


{'SPY': {'realestate': 0.028399998, 'consumer_cyclical': 0.111899994, 'basic_materials': 0.0225, 'consumer_defensive': 0.0717, 'technology': 0.2464, 'communication_services': 0.084, 'financial_services': 0.1299, 'utilities': 0.024500001, 'industrials': 0.0846, 'energy': 0.0466, 'healthcare': 0.14130001}}


,sector,date,adj_close,sector_weight,pct_change,contribution
0,S&P 500 Materials (Sector),2015-01-07,298.519989,0.0225,NaN,NaN
1,S&P 500 Materials (Sector),2015-01-08,305.600006,0.0225,0.023717,0.000534
2,S&P 500 Materials (Sector),2015-01-09,304.070007,0.0225,-0.005007,-0.000113
3,S&P 500 Materials (Sector),2015-01-12,303.130005,0.0225,-0.003091,-0.000070
4,S&P 500 Materials (Sector),2015-01-13,299.609985,0.0225,-0.011612,-0.000261
...,...,...,...,...,...,...
50,S&P 500 Energy (Sector),2015-01-07,560.090027,0.0466,NaN,NaN
51,S&P 500 Energy (Sector),2015-01-08,572.440002,0.0466,0.022050,0.001028
52,S&P 500 Energy (Sector),2015-01-09,567.919983,0.0466,-0.007896,-0.000368
53,S&P 500 Energy (Sector),2015-01-12,552.000000,0.0466,-0.028032,-0.001306


In [18]:
contributions = df.groupby("sector").agg({"contribution": "sum"})
contributions["contribution_as_pct"] = (contributions["contribution"] / df["contribution"].sum())*100
contributions

,contribution,contribution_as_pct
sector,,
S&P 500 Consumer Discretionary (Sector),-0.000275,-21.424743
S&P 500 Consumer Staples (Sector),0.000252,19.603983
S&P 500 Energy (Sector),-0.000985,-76.773076
S&P 500 Financials (Sector),-0.001363,-106.234896
S&P 500 Health Care (Sector),0.000505,39.334654
S&P 500 Industrials (Sector),0.000042,3.239738
S&P 500 Information Technology (Sector),0.001434,111.761223
S&P 500 Materials (Sector),0.000090,7.025085
S&P 500 Real Estate (Sector),0.000248,19.306254


In [ ]:
# weights

In [ ]:
# yq.Ticker(sectors_ticker).fund_sector_weightings.SPY.sum()

In [ ]:
# Starting with visualisation:
import matplotlib.pyplot as plt
fig=plt.figure(1)
plt.barh(contributions.index, contributions["contribution_as_pct"],align="center")


plt.title("S&P 500 Attributions By Sector")
plt.legend(labels=["% Attribution"])
plt.show()

In [ ]:
contributions_test = contributions.copy()
contributions_test.iloc[-1][1] = 100

In [ ]:
fig=plt.figure(1)
plt.barh(contributions_test.index, contributions_test["contribution_as_pct"],align="center")


plt.title("S&P 500 Attributions By Sector")
plt.legend(labels=["% Attribution"])
plt.show()

# Denzel: Printing individual time dataframe

So we need:

- MTD
- QTD
- YTD
- 3m
- 6m
- 1y
- 3y
- 5y
- 10y
- all

In [19]:
# Frist lets package everything into a nice little function.

def cont(start_date, end_date):# format like 2015-01-15 (YYYY-MM-DD)
    
    # Generate data frame to hold all required information
    sectors_ticker = "SPY"

    # Load in info
    sp500_tickers_data = get_req_data(start_date, end_date)
    weights = yq.Ticker(sectors_ticker).fund_sector_weightings.to_dict()

    # add the sectors + dates + adj close to the dataframe
    records = []
    for sector, data in sp500_tickers_data.items():
        for x in range(0, len(data['sector_data'])):

            record = {"sector" : sector, "date" : data['sector_data'].index[x], "adj_close" : data["sector_data"][x], "sector_weight" : weights[sectors_ticker][sector_map[sector]] }
            records.append(record)

    df = pd.DataFrame(records)

    df["pct_change"] = df.groupby("sector")["adj_close"].pct_change()

    df["contribution"] = df["pct_change"] * df["sector_weight"]

    #print(weights)
    #display(df)

    contributions = df.groupby("sector").agg({"contribution": "sum"})
    contributions["contribution_as_pct"] = (contributions["contribution"] / df["contribution"].sum())*100

    return contributions


    

In [20]:
cont("2000-1-1", date.today())

,contribution,contribution_as_pct
sector,,
S&P 500 Consumer Discretionary (Sector),0.215921,13.704779
S&P 500 Consumer Staples (Sector),0.108557,6.890251
S&P 500 Energy (Sector),0.087548,5.556768
S&P 500 Financials (Sector),0.194997,12.376735
S&P 500 Health Care (Sector),0.262609,16.668092
S&P 500 Industrials (Sector),0.128200,8.137039
S&P 500 Information Technology (Sector),0.456783,28.992610
S&P 500 Materials (Sector),0.037737,2.395205
S&P 500 Real Estate (Sector),0.053863,3.418739


### The Building begins...

In [ ]:
# MTD Builder
mtd_start = "{year}-{month}-1"

cur_month = date.today().month
cur_year = date.today().year

mtd_start_date = mtd_start.format(year=cur_year, month=cur_month)

mtd_end_date = date.today()

# Printing result
#print(mtd_start_date,mtd_end_date)
cont(mtd_start_date, mtd_end_date)

In [ ]:
#QTD builder

cm = date.today().month

if cm >= 1 and cm <= 3:
    qtd_start_date = str(date.today().year) + "-1-1" #Q1
elif cm >= 4 and cm <= 6:
    qtd_start_date = str(date.today().year) + "-4-1" #Q2
elif cm >= 7 and cm <= 9:
    qtd_start_date = str(date.today().year) + "-7-1" #Q3
elif cm >= 10 and cm <= 12:
    qtd_start_date = str(date.today().year) + "-10-1" #Q4
else:
    print("Error")
    
    
qtd_end_date = date.today()

#Printing Result
print(qtd_start_date,qtd_end_date)
cont(qtd_start_date,qtd_end_date)

In [ ]:
# YTD Builder

ytd_start_date = str(date.today().year) + "-1-1"
ytd_end_date = date.today()

print(ytd_start_date,ytd_end_date)
cont(ytd_start_date,ytd_end_date)

In [ ]:
#3M
cont(date.today() + relativedelta(months=-3), date.today())
#6M
cont(date.today() + relativedelta(months=-6), date.today())
#1y
cont(date.today() + relativedelta(years=-1), date.today())
#3y
cont(date.today() + relativedelta(years=-3), date.today())
#5y
cont(date.today() + relativedelta(years=-5), date.today())
#10y
cont(date.today() + relativedelta(years=-10), date.today())
#ALL
cont("1928-1-2", date.today()) #idk about this one... Seems too low for a 90+ year gap.

In [ ]:
result = cont("2000-1-1", date.today())
result

In [ ]:

###

#result_df["Portfolio [%]"] = # INSERT PORTFOLIO ATTRIBUTIONS HERE!!!


###

In [3]:
# NEW (Slightly editied) function that returns in desired output Dataframe 

def cont2(start_date, end_date):# format like 2015-01-15 (YYYY-MM-DD)
    
    # Generate data frame to hold all required information
    sectors_ticker = "SPY"

    # Load in info
    sp500_tickers_data = get_req_data(start_date, end_date)
    weights = yq.Ticker(sectors_ticker).fund_sector_weightings.to_dict()

    # add the sectors + dates + adj close to the dataframe
    records = []
    for sector, data in sp500_tickers_data.items():
        for x in range(0, len(data['sector_data'])):

            record = {"sector" : sector, "date" : data['sector_data'].index[x], "adj_close" : data["sector_data"][x], "sector_weight" : weights[sectors_ticker][sector_map[sector]] }
            records.append(record)

    df = pd.DataFrame(records)

    df["pct_change"] = df.groupby("sector")["adj_close"].pct_change()

    df["contribution"] = df["pct_change"] * df["sector_weight"]

    #print(weights)
    #display(df)

    contributions = df.groupby("sector").agg({"contribution": "sum"})
    contributions["contribution_as_pct"] = (contributions["contribution"] / df["contribution"].sum())*100
    
    # We standardize output DF form here
    result_df = contributions.loc[:,contributions.columns != "contribution"]
    
    result_df.rename(columns={"contribution_as_pct":"S&P 500 [%]"}, inplace=True)
    
    # result_df["Portfolio [%]"] =  #[INSERT PORTFOLIO ATTRIBUTIONS HERE!!!]

    return result_df


    

In [4]:
cont2('2010-1-1',date.today())

/Users/denzelo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,S&P 500 [%]
sector,
S&P 500 Consumer Discretionary (Sector),14.054374
S&P 500 Consumer Staples (Sector),5.368184
S&P 500 Energy (Sector),2.328244
S&P 500 Financials (Sector),11.712942
S&P 500 Health Care (Sector),14.949468
S&P 500 Industrials (Sector),7.714529
S&P 500 Information Technology (Sector),34.749827
S&P 500 Materials (Sector),1.622162
S&P 500 Real Estate (Sector),2.312026


In [5]:
sp500_tickers_data = get_req_data('2010-1-1',date.today())

In [6]:
sp500_tickers_data

{'S&P 500 Materials (Sector)': {'sector_data': Date
  2009-12-31    199.809998
  2010-01-04    205.360001
  2010-01-05    206.429993
  2010-01-06    209.570007
  2010-01-07    208.610001
                   ...    
  2022-09-19    464.260010
  2022-09-20    455.450012
  2022-09-21    445.450012
  2022-09-22    439.910004
  2022-09-23    430.899994
  Name: Adj Close, Length: 3202, dtype: float64},
 'S&P 500 Industrials (Sector)': {'sector_data': Date
  2009-12-31    242.990005
  2010-01-04    247.110001
  2010-01-05    247.990005
  2010-01-06    248.490005
  2010-01-07    251.720001
                   ...    
  2022-09-19    761.789978
  2022-09-20    752.380005
  2022-09-21    742.320007
  2022-09-22    731.270020
  2022-09-23    717.599976
  Name: Adj Close, Length: 3202, dtype: float64},
 'S&P 500 Consumer Discretionary (Sector)': {'sector_data': Date
  2009-12-31     235.070007
  2010-01-04     236.460007
  2010-01-05     237.619995
  2010-01-06     237.589996
  2010-01-07     239.63

# Categorizing Over/UnderExposure

In [12]:
result = cont2('2022-1-1',date.today())
result

/Users/denzelo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,S&P 500 [%]
sector,
S&P 500 Consumer Discretionary (Sector),15.276670
S&P 500 Consumer Staples (Sector),3.154398
S&P 500 Energy (Sector),-6.541778
S&P 500 Financials (Sector),12.687176
S&P 500 Health Care (Sector),8.324201
S&P 500 Industrials (Sector),8.136681
S&P 500 Information Technology (Sector),35.225527
S&P 500 Materials (Sector),2.730421
S&P 500 Real Estate (Sector),4.038064


In [31]:
result["Portfolio[%]"]=[14.3,4,-5,12.7,9,8,40,-1.4,3.9,15,0.1]
result

,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio
sector,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,0.988858
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809


In [32]:
# 1. Excess Attribution

result["Excess Attribution"] = result["Portfolio[%]"] - result["S&P 500 [%]"] 
result

,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio
sector,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,0.988858
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809


In [33]:
# 2. Attribution Ratio

result["Attribution Ratio"] = result["Portfolio[%]"] / result["S&P 500 [%]"] 
result

,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio
sector,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,-0.512741
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809


In [38]:
# 3. Attribution Direction

direction = []

for ratio in result["Attribution Ratio"]:
    if ratio >= 0:
        direction.append("Correlated (+)")
    elif ratio <0:
        direction.append("Uncorrelated (-)")

        
result["Attribution Direction [+/-]"] = direction

In [39]:
result

,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio,Attribution Direction [+/-]
sector,,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068,Correlated (+)
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071,Correlated (+)
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318,Correlated (+)
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011,Correlated (+)
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185,Correlated (+)
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202,Correlated (+)
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540,Correlated (+)
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,-0.512741,UnCorrelated (-)
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809,Correlated (+)


In [40]:
# 4. Attribution Sensetivity 

sensitivity = []

for ratio in result["Attribution Ratio"]:
    if abs(ratio) > 1.25:
        sensitivity.append("High")
    elif 0.75 <= abs(ratio) <= 1.25  :
        sensitivity.append("Normal")
    elif abs(ratio) < 0.75 :
        sensitivity.append("Low")

        
result["Attribution Sensitivity"] = sensitivity

In [41]:
result

,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio,Attribution Direction [+/-],Attribution Sensitivity
sector,,,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068,Correlated (+),Normal
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071,Correlated (+),High
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318,Correlated (+),Normal
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011,Correlated (+),Normal
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185,Correlated (+),Normal
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202,Correlated (+),Normal
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540,Correlated (+),Normal
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,-0.512741,UnCorrelated (-),Low
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809,Correlated (+),Normal


## Putting it all into a singular function

In [44]:
# Inputs: 
# result = df of the S&P 500 attribs
# portfolio = df (OR LIST) of the Portfolio attribs


def attrib_categorizer(result, portfolio):

    result["Portfolio[%]"]= portfolio
    
    
    # 1. Excess Attribution

    result["Excess Attribution"] = result["Portfolio[%]"] - result["S&P 500 [%]"] 
    result

    # 2. Attribution Ratio
    
    result["Attribution Ratio"] = result["Portfolio[%]"] / result["S&P 500 [%]"] 
    result

    # 3. Attribution Direction

    direction = []

    for ratio in result["Attribution Ratio"]:
        if ratio >= 0:
            direction.append("Correlated (+)")
        elif ratio <0:
            direction.append("Uncorrelated (-)")


    result["Attribution Direction [+/-]"] = direction

    
    # 4. Attribution Sensetivity 

    sensitivity = []

    for ratio in result["Attribution Ratio"]:
        if abs(ratio) > 1.25:
            sensitivity.append("High")
        elif 0.75 <= abs(ratio) <= 1.25  :
            sensitivity.append("Normal")
        elif abs(ratio) < 0.75 :
            sensitivity.append("Low")


    result["Attribution Sensitivity"] = sensitivity
    
    return result

In [45]:
ex_snp500 = cont2("2022-1-1",date.today())
ex_portfolio = [14.3,4,-5,12.7,9,8,40,-1.4,3.9,15,0.1]

attrib_categorizer(ex_snp500,ex_portfolio)

/Users/denzelo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,S&P 500 [%],Portfolio[%],Excess Attribution,Attribution Ratio,Attribution Direction [+/-],Attribution Sensitivity
sector,,,,,,
S&P 500 Consumer Discretionary (Sector),15.276670,14.3,-0.976670,0.936068,Correlated (+),Normal
S&P 500 Consumer Staples (Sector),3.154398,4.0,0.845602,1.268071,Correlated (+),High
S&P 500 Energy (Sector),-6.541778,-5.0,1.541778,0.764318,Correlated (+),Normal
S&P 500 Financials (Sector),12.687176,12.7,0.012824,1.001011,Correlated (+),Normal
S&P 500 Health Care (Sector),8.324201,9.0,0.675799,1.081185,Correlated (+),Normal
S&P 500 Industrials (Sector),8.136681,8.0,-0.136681,0.983202,Correlated (+),Normal
S&P 500 Information Technology (Sector),35.225527,40.0,4.774473,1.135540,Correlated (+),Normal
S&P 500 Materials (Sector),2.730421,-1.4,-4.130421,-0.512741,Uncorrelated (-),Low
S&P 500 Real Estate (Sector),4.038064,3.9,-0.138064,0.965809,Correlated (+),Normal
